In [15]:
import pandas as pd
from g4f.client import Client

In [16]:
def load_data(file_path: str) -> pd.DataFrame:
    """
    加載 CSV 文件並處理錯誤行。
    """
    try:
        return pd.read_csv(file_path, on_bad_lines="skip", encoding="utf-8")
    except pd.errors.ParserError as e:
        raise Exception(f"CSV 文件解析錯誤：{e}")
    except Exception as e:
        raise Exception(f"讀取數據時發生錯誤：{e}")



In [17]:
def generate_summary(data: pd.DataFrame) -> pd.DataFrame:
    """
    生成數據描述性統計。
    Args:
        data (pd.DataFrame): 輸入數據框。
    Returns:
        pd.DataFrame: 描述性統計。
    """
    try:
        return data.describe()
    except Exception as e:
        raise Exception(f"生成描述性統計時發生錯誤：{e}")

In [28]:
def construct_prompt(user_input: str, summary_stats: pd.DataFrame) -> tuple:
    """
    構建系統與用戶提示。
    Args:
        user_input (str): 用戶查詢。
        summary_stats (pd.DataFrame): 描述性統計。
    Returns:
        tuple: (系統提示, 用戶提示)
    """
    system_prompt =( 
    
    """
        You are a professional data scientist skilled in extracting valuable insights from data. 
        Based on the provided descriptive statistics, perform an in-depth analysis and generate actionable recommendations in Chinese:
        1. Summarize the overall characteristics of the data, such as mean, standard deviation, and distribution.
        2. Identify potential outliers or data quality issues, and evaluate their impact on the analysis.
        3. If there are categorical variables, analyze their distribution and possible correlations with other features.
        4. Provide actionable recommendations based on the data, such as potential improvement areas or specific domains that require attention.
        5. Clearly and concisely communicate your conclusions, and suggest next steps for further data collection or analysis.

        The final report should be written in tranditional Chinese.
        """
    )

    user_prompt = (
        f"以下是數據的描述性統計結果：\n{summary_stats.to_string()}\n"
        f"請根據這些數據回答以下問題：{user_input}"
    )
    return system_prompt, user_prompt


In [29]:
def generate_report(file_path: str, user_input: str) -> str:
    """
    基於自然語言查詢生成分析報告。
    Args:
        file_path (str): 數據文件路徑。
        user_input (str): 用戶的自然語言查詢。
    Returns:
        str: 分析報告或錯誤信息。
    """
    try:
        # 1. 加載數據
        data = load_data(file_path)

        # 2. 生成描述性統計
        summary_stats = generate_summary(data)

        # 3. 構建 GPT 模型請求的提示
        system_prompt, user_prompt = construct_prompt(user_input, summary_stats)

        # 4. 初始化 GPT 客戶端並生成報告
        client = Client()
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.1,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"生成報告時發生錯誤：{e}"



In [30]:
if __name__ == "__main__":
    # 示例使用
    file_path = r".\csv.ingredient\test.csv"


    user_query = "請分析數據並提供詳細的見解和建議。"  # 自然語言查詢

    try:
        report = generate_report(file_path, user_query)
        print("生成的報告：")
        print(report)
    except Exception as e:
        print(f"程序運行失敗：{e}")

生成的報告：
根據提供的數據描述性統計結果，本報告將對數據進行深入分析並提出可行建議。

### 1. 整體數據特徵總結
根據數據，收集的樣本數為60個年份的本益比（P/E Ratio）。以下是該數據的主要特徵：

- **年份**：數據範圍從2010到2024年，顯示了十五年的趨勢。
- **本益比（P/E Ratio）**：
  - **平均值**：20.463，這表明整體市場可能被輕微高估，因為這一數值超過了20，通常被視為警示線。
  - **標準差**：3.6059，表示P/E Ratio的波動相對適中，數據的分佈趨於集中。
  - **最小值與最大值**：P/E Ratio範圍從14.11到26.65，最高的P/E Ratio顯示了某些年份的市場熱情或估值偏高。

### 2. 潛在異常值和數據質量問題識別
通過查看該數據的最大值和最小值，我們可以注意到P/E Ratio的最小值為14.11，而最大值為26.65。這可能表明：

- **潛在異常值**：如果在數據中存在極端高或極端低的P/E Ratio，則可能會影響整體統計結果。需要進一步調查這些高值的年份是否存在特殊市場變化或經濟事件。
- **數據質量**：建議確認所有數據的來源，以確保其可靠性。此外，檢查是否存在缺失值或錄入錯誤。

### 3. 類別變數的分佈及潛在關聯分析
在這些數據中目前僅有年份和P/E Ratio這兩個數據類型，假如我們能得到更多類別變數（如行業類別），將能夠進行更深入的分析和關聯檢測。例如：

- 分析各行業的P/E Ratio平均值，判斷不同行業的市場表現。
- 探討年份與P/E Ratio之間的可能趨勢，例如是否在某些特定年份中整體的P/E Ratio有顯著上升或下降的情況。

### 4. 可行建議
基於以上分析，提出以下幾點建議：

- **監控市場趨勢**：持續評估P/E Ratio的變化以洞悉市場情緒，尤其是在接下來的年份，識別可能的高風險年份。
- **深入行業分析**：若可行，收集不同行業的相關數據以進行更細緻的比較，這能幫助投資者和決策者了解哪些行業在特定期間內表現良好或不佳。
- **評估影響因素**：深入研究市場中影響P/E Ratio的重要因子，如利潤增長、經濟環境變化等，這有助於理解市場動態。

### 5. 總結及後續步驟
總的來說，本益比數據